In [3]:
!pip install sqlalchemy

In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('sqlite:///test.db',echo = True)

In [ ]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

metadata = MetaData()

users = Table('users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('fullname', String),
)

addresses = Table('addresses', metadata,
  Column('id', Integer, primary_key=True),
  Column('user_id', None, ForeignKey('users.id')),
  Column('email_address', String, nullable=False)
 )

In [8]:
metadata.create_all(engine)

2019-12-19 15:27:39,625 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-12-19 15:27:39,627 INFO sqlalchemy.engine.base.Engine ()
2019-12-19 15:27:39,632 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-12-19 15:27:39,634 INFO sqlalchemy.engine.base.Engine ()
2019-12-19 15:27:39,639 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2019-12-19 15:27:39,640 INFO sqlalchemy.engine.base.Engine ()
2019-12-19 15:27:39,644 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2019-12-19 15:27:39,645 INFO sqlalchemy.engine.base.Engine ()
2019-12-19 15:27:39,647 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("addresses")
2019-12-19 15:27:39,648 INFO sqlalchemy.engine.base.Engine ()
2019-12-19 15:27:39,649 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("addresses")
2019-12-19 15:27:39,651 INFO sqlalchemy.engine.base.Engine ()
2019-12-19 15:27:3

In [ ]:
ins = users.insert()

In [ ]:
ins = users.insert().values(name='jack', fullname='Jack Jones')

In [12]:
ins.compile().params

{'fullname': 'Jack Jones', 'name': 'jack'}

In [ ]:
conn = engine.connect()

In [14]:
result = conn.execute(ins)
result

2019-12-19 15:33:48,402 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-12-19 15:33:48,405 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Jones')
2019-12-19 15:33:48,408 INFO sqlalchemy.engine.base.Engine COMMIT


In [15]:
ins.bind = engine
str(ins)

'INSERT INTO users (name, fullname) VALUES (?, ?)'

In [16]:
result.inserted_primary_key

[1]

In [18]:
ins = users.insert()
conn.execute(ins, id=2, name='wendy', fullname='Wendy Williams')

2019-12-19 15:36:34,824 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname) VALUES (?, ?, ?)
2019-12-19 15:36:34,826 INFO sqlalchemy.engine.base.Engine (2, 'wendy', 'Wendy Williams')
2019-12-19 15:36:34,829 INFO sqlalchemy.engine.base.Engine COMMIT


In [19]:
conn.execute(addresses.insert(), [
   {'user_id': 1, 'email_address' : 'jack@yahoo.com'},
   {'user_id': 1, 'email_address' : 'jack@msn.com'},
   {'user_id': 2, 'email_address' : 'www@www.org'},
   {'user_id': 2, 'email_address' : 'wendy@aol.com'},
])

2019-12-19 15:36:59,019 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2019-12-19 15:36:59,020 INFO sqlalchemy.engine.base.Engine ((1, 'jack@yahoo.com'), (1, 'jack@msn.com'), (2, 'www@www.org'), (2, 'wendy@aol.com'))
2019-12-19 15:36:59,023 INFO sqlalchemy.engine.base.Engine COMMIT


In [21]:
from sqlalchemy.sql import select
s = select([users])
result = conn.execute(s)

2019-12-19 15:38:00,736 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-12-19 15:38:00,738 INFO sqlalchemy.engine.base.Engine ()


In [22]:
for row in result:
    print(row)

(1, 'jack', 'Jack Jones')
(2, 'wendy', 'Wendy Williams')


In [23]:
result = conn.execute(s)
row = result.fetchone()
print("name:", row['name'], "; fullname:", row['fullname'])

2019-12-19 15:40:58,801 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-12-19 15:40:58,803 INFO sqlalchemy.engine.base.Engine ()
name: jack ; fullname: Jack Jones


In [24]:
row = result.fetchone()
print("name:", row[1], "; fullname:", row[2])

name: wendy ; fullname: Wendy Williams


In [25]:
for row in conn.execute(s):
    print("name:", row[users.c.name], "; fullname:", row[users.c.fullname])

2019-12-19 15:44:26,322 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-12-19 15:44:26,324 INFO sqlalchemy.engine.base.Engine ()
name: jack ; fullname: Jack Jones
name: wendy ; fullname: Wendy Williams


In [ ]:
result.close()

In [27]:
s = select([users.c.name, users.c.fullname])
result = conn.execute(s)
for row in result:
    print(row)

2019-12-19 15:45:25,488 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2019-12-19 15:45:25,492 INFO sqlalchemy.engine.base.Engine ()
('jack', 'Jack Jones')
('wendy', 'Wendy Williams')


In [28]:
s = select([users, addresses]).where(users.c.id == addresses.c.user_id)
for row in conn.execute(s):
    print(row)

2019-12-19 15:46:19,286 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2019-12-19 15:46:19,287 INFO sqlalchemy.engine.base.Engine ()
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')
(2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org')
(2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')
